In [2]:
# ---------------------------------------------------
# 🧠 AlexNet (2012) Implementation on CIFAR-10 Dataset
# ---------------------------------------------------
# ✔️ Deep CNN architecture that revolutionized image classification
# ✔️ Trained on CIFAR-10 (10 classes, 32x32 color images)
# ✔️ Fully annotated for learning and modification
# ---------------------------------------------------

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

# ✅ Use GPU if available for faster training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------------------
# 🔄 Data Transformation and Preprocessing
# -----------------------------------------
# CIFAR-10 images are 32x32 — we upscale them to 224x224 for AlexNet
# Normalize images to mean 0 and std 1 for each RGB channel
transform = transforms.Compose([
    transforms.Resize((224, 224)),             # Resize to 224x224 (AlexNet input size)
    transforms.ToTensor(),                     # Convert PIL image to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5),       # Normalize RGB channels
                         (0.5, 0.5, 0.5))
])

# 📥 Download CIFAR-10 dataset and prepare loaders
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# -----------------------------------
# 🏗️ AlexNet Model Architecture Class
# -----------------------------------
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):  # CIFAR-10 has 10 classes
        super(AlexNet, self).__init__()

        # 🧱 Feature extractor: 5 convolutional layers + ReLU + MaxPooling
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),  # Conv1
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # Pool1

            nn.Conv2d(96, 256, kernel_size=5, padding=2),           # Conv2
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # Pool2

            nn.Conv2d(256, 384, kernel_size=3, padding=1),          # Conv3
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, padding=1),          # Conv4
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),          # Conv5
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)                   # Pool3
        )

        # 🧠 Classifier: 3 fully connected layers + dropout
        self.classifier = nn.Sequential(
            nn.Dropout(),                                           # Dropout1
            nn.Linear(256 * 6 * 6, 4096),                           # FC1
            nn.ReLU(inplace=True),

            nn.Dropout(),                                           # Dropout2
            nn.Linear(4096, 4096),                                  # FC2
            nn.ReLU(inplace=True),

            nn.Linear(4096, num_classes)                            # FC3 (output)
        )

    def forward(self, x):
        x = self.features(x)               # Apply all convolutional layers
        x = torch.flatten(x, 1)            # Flatten for FC layers
        x = self.classifier(x)             # Apply all fully connected layers
        return x

# 🔧 Instantiate the model and move it to device
model = AlexNet().to(device)

# --------------------------
# 🎯 Loss & Optimizer Setup
# --------------------------
criterion = nn.CrossEntropyLoss()           # Good for classification tasks
optimizer = optim.Adam(model.parameters(),  # Adam optimizer (adaptive LR)
                       lr=0.001)

# ----------------------------
# 🔁 Model Training Procedure
# ----------------------------
num_epochs = 5  # Increase to 5+ for better accuracy

print("Starting training...\n")
start_time = time.time()

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()             # 1. Reset gradients
        outputs = model(images)           # 2. Forward pass
        loss = criterion(outputs, labels) # 3. Compute loss
        loss.backward()                   # 4. Backpropagation
        optimizer.step()                  # 5. Update weights

        running_loss += loss.item()

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], "
                  f"Step [{batch_idx+1}/{len(train_loader)}], "
                  f"Loss: {loss.item():.4f}")

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}] completed. Average loss: {avg_loss:.4f}")

print(f"\n✅ Training complete in {(time.time() - start_time):.2f} seconds")

# ---------------------------
# 🧪 Model Evaluation (Test)
# ---------------------------
print("\nEvaluating on test set...")

model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradients for faster inference
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)  # Get predicted class (max score)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 📊 Final Accuracy Report
accuracy = 100 * correct / total
print(f"✅ Test Accuracy: {accuracy:.2f}%")


Using device: cuda
Starting training...

Epoch [1/5], Step [100/782], Loss: 2.1592
Epoch [1/5], Step [200/782], Loss: 1.9989
Epoch [1/5], Step [300/782], Loss: 1.8247
Epoch [1/5], Step [400/782], Loss: 1.6186
Epoch [1/5], Step [500/782], Loss: 1.8480
Epoch [1/5], Step [600/782], Loss: 1.7065
Epoch [1/5], Step [700/782], Loss: 1.5290
Epoch [1] completed. Average loss: 1.7944
Epoch [2/5], Step [100/782], Loss: 1.2630
Epoch [2/5], Step [200/782], Loss: 1.3353
Epoch [2/5], Step [300/782], Loss: 1.6964
Epoch [2/5], Step [400/782], Loss: 1.3499
Epoch [2/5], Step [500/782], Loss: 1.3556
Epoch [2/5], Step [600/782], Loss: 1.4121
Epoch [2/5], Step [700/782], Loss: 1.2016
Epoch [2] completed. Average loss: 1.4314
Epoch [3/5], Step [100/782], Loss: 1.4352
Epoch [3/5], Step [200/782], Loss: 1.2223
Epoch [3/5], Step [300/782], Loss: 1.2714
Epoch [3/5], Step [400/782], Loss: 1.2867
Epoch [3/5], Step [500/782], Loss: 1.4118
Epoch [3/5], Step [600/782], Loss: 1.4708
Epoch [3/5], Step [700/782], Loss: 